In [11]:
# Backtesting strategy: Intraday resistance breakout strategy

import numpy as np
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import copy
import time

In [12]:
def ATR(DF,n):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    #df['ATR'] = df['TR'].ewm(span=n,adjust=False,min_periods=n).mean()
    df2 = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df2['ATR']

def CAGR(DF):
    "function to calculate the Cumulative Annual Growth Rate of a trading strategy"
    df = DF.copy()
    df["cum_return"] = (1 + df["ret"]).cumprod()
    
    # We will use intraday data. In this case we are using 5 minute candles. 
    # In each trading day we will have 78 five minute candles. 1/(252*78) corresponds to 1/year again.
    n = len(df)/(252*78)  
    CAGR = (df["cum_return"].tolist()[-1])**(1/n) - 1
    return CAGR

def volatility(DF):
    "function to calculate annualized volatility of a trading strategy"
    df = DF.copy()
    vol = df["ret"].std() * np.sqrt(252*78)
    return vol

def sharpe(DF,rf):
    "function to calculate sharpe ratio ; rf is the risk free rate"
    df = DF.copy()
    sr = (CAGR(df) - rf)/volatility(df)
    return sr
    
def max_dd(DF):
    "function to calculate max drawdown"
    df = DF.copy()
    df["cum_return"] = (1 + df["ret"]).cumprod()
    df["cum_roll_max"] = df["cum_return"].cummax()
    df["drawdown"] = df["cum_roll_max"] - df["cum_return"]
    df["drawdown_pct"] = df["drawdown"]/df["cum_roll_max"]
    max_dd = df["drawdown_pct"].max()
    return max_dd

In [4]:
# Download historical data (monthly) for selected stocks

tickers = ["MSFT","AAPL","FB","AMZN","INTC", "CSCO","VZ","IBM","TSLA","AMD"]
        
ts = TimeSeries(key='25V9C3YS6OPO9YSE', output_format='pandas') # key=open(key_path,'r').read()

In [5]:
ohlc_intraday = {} # directory with ohlc value for each stock   
api_call_count = 1
ts = TimeSeries(key='25V9C3YS6OPO9YSE', output_format='pandas') # key=open(key_path,'r').read()
start_time = time.time()

In [6]:
for ticker in tickers:
    data = ts.get_intraday(symbol=ticker,interval='5min', outputsize='full')[0]
    api_call_count+=1
    data.columns = ["Open","High","Low","Close","Volume"]
    data = data.iloc[::-1]
    data = data.between_time('09:35', '16:00') #remove data outside regular trading hours
    ohlc_intraday[ticker] = data
    if api_call_count==5:
        api_call_count = 1
        time.sleep(60 - ((time.time() - start_time) % 60.0))

In [8]:
tickers = ohlc_intraday.keys() # redefine tickers variable after removing any tickers with corrupted data
tickers

dict_keys(['MSFT', 'AAPL', 'FB', 'AMZN', 'INTC', 'CSCO', 'VZ', 'IBM', 'TSLA', 'AMD'])

In [9]:
################################Backtesting####################################

# calculating ATR and rolling max price for each stock and consolidating this info by stock in a separate dataframe
ohlc_dict = copy.deepcopy(ohlc_intraday)
tickers_signal = {}
tickers_ret = {}
for ticker in tickers:
    print("calculating ATR and rolling max price for ",ticker)
    ohlc_dict[ticker]["ATR"] = ATR(ohlc_dict[ticker],20)
    ohlc_dict[ticker]["roll_max_cp"] = ohlc_dict[ticker]["High"].rolling(20).max()
    ohlc_dict[ticker]["roll_min_cp"] = ohlc_dict[ticker]["Low"].rolling(20).min()
    ohlc_dict[ticker]["roll_max_vol"] = ohlc_dict[ticker]["Volume"].rolling(20).max()
    ohlc_dict[ticker].dropna(inplace=True)
    tickers_signal[ticker] = ""
    tickers_ret[ticker] = [0]

calculating ATR and rolling max price for  MSFT
calculating ATR and rolling max price for  AAPL
calculating ATR and rolling max price for  FB
calculating ATR and rolling max price for  AMZN
calculating ATR and rolling max price for  INTC
calculating ATR and rolling max price for  CSCO
calculating ATR and rolling max price for  VZ
calculating ATR and rolling max price for  IBM
calculating ATR and rolling max price for  TSLA
calculating ATR and rolling max price for  AMD


In [10]:
ohlc_dict

{'MSFT':                          Open      High       Low     Close     Volume  \
 date                                                                     
 2021-04-06 11:15:00  248.5400  248.9300  248.5200  248.8900   211688.0   
 2021-04-06 11:20:00  248.8850  249.1268  248.6300  248.7800   261704.0   
 2021-04-06 11:25:00  248.7900  248.8600  248.4441  248.4500   150687.0   
 2021-04-06 11:30:00  248.4600  248.4800  248.2700  248.3800   164307.0   
 2021-04-06 11:35:00  248.3701  248.7000  248.3132  248.5181   214457.0   
 ...                       ...       ...       ...       ...        ...   
 2021-05-04 15:40:00  246.9200  247.3400  246.9200  247.2800   377541.0   
 2021-05-04 15:45:00  247.2984  247.3100  246.8800  246.9400   302397.0   
 2021-05-04 15:50:00  246.9450  247.0400  246.7300  246.8950   362908.0   
 2021-05-04 15:55:00  246.9000  247.4200  246.7400  247.0100  2251989.0   
 2021-05-04 16:00:00  247.0050  248.0000  246.9000  247.9300  1716151.0   
 
               

In [13]:
tickers_signal

{'MSFT': '',
 'AAPL': '',
 'FB': '',
 'AMZN': '',
 'INTC': '',
 'CSCO': '',
 'VZ': '',
 'IBM': '',
 'TSLA': '',
 'AMD': ''}

In [14]:
tickers_ret

{'MSFT': [0],
 'AAPL': [0],
 'FB': [0],
 'AMZN': [0],
 'INTC': [0],
 'CSCO': [0],
 'VZ': [0],
 'IBM': [0],
 'TSLA': [0],
 'AMD': [0]}